In [1]:
from datetime import datetime as dt
import datetime
from datetime import datetime 
from datetime import date, timedelta	
import time
import gc
import cx_Oracle

from calendar import monthrange

import warnings
warnings.filterwarnings("ignore")
## To assign session Id
import uuid

import requests
from bs4 import BeautifulSoup

import numpy as np
import pandas as pd
# from layouts import *

from components import *
from config import *
## For session and enhance performance
# from os import path
# import sys
# sys.path.append(path.join(path.dirname(__file__), '..'))
from app import cache1, TIMEOUT, session_id
import locale
from locale import atof
locale.setlocale(locale.LC_NUMERIC, '')


'English_United States.1252'

In [2]:
## DB parameters
userid ="T002CNSH"
password="Changeme01"
host="192.168.186.25"
port = "1521"
SID = "VISIONRW"

## Get connection to the DB
dsn_tns = cx_Oracle.makedsn(host, port, service_name=SID) 
conn = cx_Oracle.connect(user=userid, password=password, dsn=dsn_tns) #FIN_HISTORY_HEADERS, FRL_EXPANDED

In [3]:
#Profitability at current date
query_pl="""
    with dates as (
select LPAD(extract(day from to_date(sysdate-1)), 2, '0') as day,TO_CHAR(to_date(sysdate-1), 'YYYYMM') as year_month from dual
),

day_balance as (
select year_month,country,le_book,Sequence_FD,balance as balance,column_,dr_cr_bal_ind,bal_type
    from VISIONRW.Fin_Dly_Balances 
    unpivot ( 
        balance for column_
        in (balance_01, balance_02, balance_03, balance_04, balance_05, balance_06, balance_07, balance_08, balance_09, balance_10, balance_11, balance_12, balance_13, balance_14, balance_15, balance_16, balance_17, balance_18, balance_19, balance_20, balance_21, balance_22, balance_23, balance_24, balance_25, balance_26, balance_27, balance_28, balance_29, balance_30, balance_31)
        ) balance_unpivot 
        where year_month = (select year_month from dates fetch next 1 row only) and column_ between 'BALANCE_01' and CONCAT('BALANCE_',(select day from dates fetch next 1 row only))
),
default_pl as (SELECT
    fin_dly_headers.year_month             AS year_month,
    fin_dly_headers.vision_sbu             AS vision_sbu,
    accounts_dly.account_type              AS ACCOUNT_TYPE,
    frl_lines_pl.frl_attribute_04          AS frl_attribute_04,
    frl_lines_pl.frl_attribute_02          AS frl_attribute_02,
    product_expanded.product_description   AS product_description,
    frl_lines_bs.source_type               AS source_type,
    fin_dly_headers.customer_id            AS customer_id,
    fin_dly_headers.bs_gl                  AS bs_gl,
    fin_dly_headers.pl_gl                  AS pl_gl,
    fin_dly_headers.currency               AS currency,
    fin_dly_headers.record_type            AS record_type,
    fin_dly_balances.bal_type              AS bal_type,
    fin_dly_headers.contract_id            AS contract_id,
    fin_dly_headers.office_account         AS office_account,
    to_number(SUM(fin_dly_balances.balance)) AS balance

FROM
    visionrw.fin_dly_headers             fin_dly_headers
    JOIN day_balance            fin_dly_balances ON fin_dly_headers.country = fin_dly_balances.country
                                              AND fin_dly_headers.le_book = fin_dly_balances.le_book
                                              AND fin_dly_headers.year_month = fin_dly_balances.year_month
                                              AND fin_dly_headers.sequence_fd = fin_dly_balances.sequence_fd
                                              AND fin_dly_headers.record_type != 9999
            LEFT JOIN visionrw.accounts_dly_view           accounts_dly ON fin_dly_headers.country = accounts_dly.country
                                                    AND fin_dly_headers.le_book = accounts_dly.le_book
                                                    AND (
            CASE
                WHEN fin_dly_headers.contract_id = '0' THEN
                    fin_dly_headers.office_account
                ELSE
                    fin_dly_headers.contract_id
            END
        ) = accounts_dly.account_no
                                                    AND fin_dly_headers.record_type != 9999                                                 
    LEFT JOIN visionrw.vw_customers_dly_expanded   customers_dly_expanded ON customers_dly_expanded.country = fin_dly_headers.country
                                                                  AND customers_dly_expanded.le_book = fin_dly_headers.le_book
                                                                  AND customers_dly_expanded.customer_id = fin_dly_headers.customer_id
                                                                  AND fin_dly_headers.record_type != 9999
    JOIN visionrw.fin_dly_mappings            fin_dly_mappings ON fin_dly_balances.country = fin_dly_mappings.country
                                              AND fin_dly_balances.le_book = fin_dly_mappings.le_book
                                              AND fin_dly_balances.year_month = fin_dly_mappings.year_month
                                              AND fin_dly_balances.sequence_fd = fin_dly_mappings.sequence_fd
                                              AND fin_dly_balances.dr_cr_bal_ind = fin_dly_mappings.dr_cr_bal_ind
    JOIN visionrw.product_expanded            product_expanded ON product_expanded.product = fin_dly_mappings.product
    JOIN visionrw.frl_lines                   frl_lines_bs ON fin_dly_mappings.frl_line_bs = frl_lines_bs.frl_line
    JOIN visionrw.frl_lines                   frl_lines_pl ON fin_dly_mappings.frl_line_pl = frl_lines_pl.frl_line
WHERE
    fin_dly_headers.year_month = (select year_month from dates fetch next 1 row only)
    AND frl_lines_bs.source_type = '0'
    AND fin_dly_headers.record_type NOT IN (
        '9999'
    )
    AND fin_dly_balances.bal_type = '54'
GROUP BY
    fin_dly_headers.year_month,
    frl_lines_pl.frl_attribute_04,
    accounts_dly.account_type,
    fin_dly_headers.vision_sbu,
    frl_lines_pl.frl_attribute_02,
    fin_dly_headers.contract_id,
    fin_dly_headers.office_account,
    product_expanded.product_description,    
    frl_lines_bs.source_type,
    fin_dly_headers.customer_id,
    fin_dly_headers.bs_gl,
    fin_dly_headers.pl_gl,
    fin_dly_headers.currency,
    fin_dly_headers.record_type,
    fin_dly_balances.bal_type),
SBU_Cases AS(
SELECT PL.*,
CASE 
WHEN CUSTOMER_ID='20002341' OR CUSTOMER_ID='20015296' OR CUSTOMER_ID='20015836'
THEN 'R'
ELSE VISION_SBU
END AS SBU
FROM default_pl PL
)

select * from SBU_Cases  Sbu

"""
PL = pd.read_sql(query_pl, con=conn)
#     PL=pd.merge(pl,accounts, on='CUSTOMER_ID', how='left')     
PL_csv = PL.to_csv(r'C:\Users\clement nshimiyima\Desktop\dt\Profitability.csv',index=None,header=True) 
#     AND frl_lines_pl.frl_attribute_02 NOT IN('FA021610','FA026025','FA021620','FA026110','FA021510',
#         'FA021430','FA022270','FA022230','FA026010','FA022260',
#          'FA022290','FA021410','FA024033','FA022320')

In [ ]:
#Removing some rows containing 
PL=PL.set_index("FRL_ATTRIBUTE_02")


In [4]:
#Profitability at current date
query_pl="""
    with dates as (
select LPAD(extract(day from to_date(sysdate-1)), 2, '0') as day,TO_CHAR(to_date(sysdate-1), 'YYYYMM') as year_month from dual
),

day_balance as (
select year_month,country,le_book,Sequence_FD,balance as balance,column_,dr_cr_bal_ind,bal_type
    from VISIONRW.Fin_Dly_Balances 
    unpivot ( 
        balance for column_
        in (balance_01, balance_02, balance_03, balance_04, balance_05, balance_06, balance_07, balance_08, balance_09, balance_10, balance_11, balance_12, balance_13, balance_14, balance_15, balance_16, balance_17, balance_18, balance_19, balance_20, balance_21, balance_22, balance_23, balance_24, balance_25, balance_26, balance_27, balance_28, balance_29, balance_30, balance_31)
        ) balance_unpivot 
        where year_month = (select year_month from dates fetch next 1 row only) and column_ between 'BALANCE_01' and CONCAT('BALANCE_',(select day from dates fetch next 1 row only))
),
default_pl as (SELECT
    fin_dly_headers.year_month             AS year_month,
    fin_dly_headers.vision_sbu             AS vision_sbu,
    accounts_dly.account_type              AS ACCOUNT_TYPE,
    frl_lines_pl.frl_attribute_04          AS frl_attribute_04,
    frl_lines_pl.frl_attribute_02          AS frl_attribute_02,
    product_expanded.product_description   AS product_description,
    frl_lines_bs.source_type               AS source_type,
    fin_dly_headers.customer_id            AS customer_id,
    fin_dly_headers.bs_gl                  AS bs_gl,
    fin_dly_headers.pl_gl                  AS pl_gl,
    fin_dly_headers.currency               AS currency,
    fin_dly_headers.record_type            AS record_type,
    fin_dly_balances.bal_type              AS bal_type,
    fin_dly_headers.contract_id            AS contract_id,
    fin_dly_headers.office_account         AS office_account,
    to_number(SUM(fin_dly_balances.balance)) AS balance

FROM
    visionrw.fin_dly_headers             fin_dly_headers
    JOIN day_balance            fin_dly_balances ON fin_dly_headers.country = fin_dly_balances.country
                                              AND fin_dly_headers.le_book = fin_dly_balances.le_book
                                              AND fin_dly_headers.year_month = fin_dly_balances.year_month
                                              AND fin_dly_headers.sequence_fd = fin_dly_balances.sequence_fd
                                              AND fin_dly_headers.record_type != 9999
            LEFT JOIN visionrw.accounts_dly_view           accounts_dly ON fin_dly_headers.country = accounts_dly.country
                                                    AND fin_dly_headers.le_book = accounts_dly.le_book
                                                    AND (
            CASE
                WHEN fin_dly_headers.contract_id = '0' THEN
                    fin_dly_headers.office_account
                ELSE
                    fin_dly_headers.contract_id
            END
        ) = accounts_dly.account_no
                                                    AND fin_dly_headers.record_type != 9999                                                 
    LEFT JOIN visionrw.vw_customers_dly_expanded   customers_dly_expanded ON customers_dly_expanded.country = fin_dly_headers.country
                                                                  AND customers_dly_expanded.le_book = fin_dly_headers.le_book
                                                                  AND customers_dly_expanded.customer_id = fin_dly_headers.customer_id
                                                                  AND fin_dly_headers.record_type != 9999
    JOIN visionrw.fin_dly_mappings            fin_dly_mappings ON fin_dly_balances.country = fin_dly_mappings.country
                                              AND fin_dly_balances.le_book = fin_dly_mappings.le_book
                                              AND fin_dly_balances.year_month = fin_dly_mappings.year_month
                                              AND fin_dly_balances.sequence_fd = fin_dly_mappings.sequence_fd
                                              AND fin_dly_balances.dr_cr_bal_ind = fin_dly_mappings.dr_cr_bal_ind
    JOIN visionrw.product_expanded            product_expanded ON product_expanded.product = fin_dly_mappings.product
    JOIN visionrw.frl_lines                   frl_lines_bs ON fin_dly_mappings.frl_line_bs = frl_lines_bs.frl_line
    JOIN visionrw.frl_lines                   frl_lines_pl ON fin_dly_mappings.frl_line_pl = frl_lines_pl.frl_line
WHERE
    fin_dly_headers.year_month = (select year_month from dates fetch next 1 row only)
    AND frl_lines_bs.source_type = '0'
    AND fin_dly_headers.record_type NOT IN (
        '9999'
    )
    AND fin_dly_balances.bal_type = '54'
GROUP BY
    fin_dly_headers.year_month,
    frl_lines_pl.frl_attribute_04,
    accounts_dly.account_type,
    fin_dly_headers.vision_sbu,
    frl_lines_pl.frl_attribute_02,
    fin_dly_headers.contract_id,
    fin_dly_headers.office_account,
    product_expanded.product_description,    
    frl_lines_bs.source_type,
    fin_dly_headers.customer_id,
    fin_dly_headers.bs_gl,
    fin_dly_headers.pl_gl,
    fin_dly_headers.currency,
    fin_dly_headers.record_type,
    fin_dly_balances.bal_type),
SBU_Cases AS(
SELECT PL.*,
CASE 
WHEN CUSTOMER_ID='20002341' OR CUSTOMER_ID='20015296' OR CUSTOMER_ID='20015836'
THEN 'R'
ELSE VISION_SBU
END AS SBU
FROM default_pl PL
)

select * from SBU_Cases  Sbu

"""
PL = pd.read_sql(query_pl, con=conn)
#     PL=pd.merge(pl,accounts, on='CUSTOMER_ID', how='left')     
PL_csv = PL.to_csv(r'C:\Users\clement nshimiyima\Desktop\dt\Profitability.csv',index=None,header=True) 
#     AND frl_lines_pl.frl_attribute_02 NOT IN('FA021610','FA026025','FA021620','FA026110','FA021510',
#         'FA021430','FA022270','FA022230','FA026010','FA022260',
#          'FA022290','FA021410','FA024033','FA022320')

In [15]:
import pandas as pd
pl=pd.read_csv(r'C:\Users\Administrator\Downloads\imdas_new\Data_im\Profitability.csv', index_col="FRL_ATTRIBUTE_02")


In [21]:
pl=pd.DataFrame(pl)
pl.head()

,YEAR_MONTH,VISION_SBU,ACCOUNT_TYPE,FRL_ATTRIBUTE_04,PRODUCT_DESCRIPTION,SOURCE_TYPE,CUSTOMER_ID,BS_GL,PL_GL,CURRENCY,RECORD_TYPE,BAL_TYPE,CONTRACT_ID,OFFICE_ACCOUNT,BALANCE,SBU
FRL_ATTRIBUTE_02,,,,,,,,,,,,,,,,
FA021410,202005,NaN,NaN,FA040100,Income Accounts,0,FT000000079,0,51001,RWF,2,54,1526676,20980251001001,653425.0,NaN
FA021430,202005,NaN,NaN,FA040100,Income Accounts,0,FT000000076,0,51009,RWF,2,54,1346702,20980251009001,786814.0,NaN
FA021430,202005,NaN,NaN,FA040100,Income Accounts,0,FT000000076,0,51009,RWF,2,54,1408207,20980251009001,776462.0,NaN
FA021430,202005,NaN,NaN,FA040100,Income Accounts,0,FT000000939,0,51009,RWF,2,54,1489462,20980251009001,621166.0,NaN
FA021430,202005,NaN,NaN,FA040100,Income Accounts,0,FT000000097,0,51011,RWF,2,54,1499513,20980251011001,465879.0,NaN


In [31]:
# pl.drop(index=['FA021610','FA026025','FA021620','FA026110','FA021510','FA021430','FA022270','FA022230','FA026010','FA022260',
# 'FA022290','FA021410','FA024033','FA022320'],inplace=True)

In [3]:
pl.FRL_ATTRIBUTE_02.value_counts()

FA022020    10652
FA021040     5188
FA024040     5095
FA021050      988
FA024015      649
FA022010      522
FA021010      380
FA024012      252
FA022060       73
FA021610       49
FA021620       40
FA026025       36
FA024010       26
FA021020       14
FA021430       14
FA021510       13
FA022270       13
FA022230        6
FA021410        3
FA026110        3
FA022260        3
FA024033        1
FA026010        1
Name: FRL_ATTRIBUTE_02, dtype: int64

In [11]:
###################Removing other income such as bonds, treasury bonds,...
pl1=pl.set_index('FRL_ATTRIBUTE_02')
# pl1.drop(['FA021610','FA026025','FA021620','FA026110','FA021510','FA021430','FA022270','FA022230','FA026010','FA022260',
# 'FA022290','FA021410','FA024033','FA022320'], inplace=True, axis=1)

In [12]:
# pl.drop(index=['FA021610','FA026025','FA021620','FA026110','FA021510','FA021430','FA022270','FA022230','FA026010','FA022260',
# 'FA022290','FA021410','FA024033','FA022320'], inplace=True, axis=1)

In [5]:
#Income
query_inc="""
    with dates as (
--  select LPAD(extract(day from to_date('20200131','yyyymmdd')), 2, '0') as day,TO_CHAR(to_date('20200131','yyyymmdd'), 'YYYYMM') as year_month from dual
select LPAD(extract(day from to_date(sysdate-1)), 2, '0') as day,TO_CHAR(to_date(sysdate-1), 'YYYYMM') as year_month , TO_CHAR(to_date(sysdate-1), 'MM') as current_month,
TO_CHAR(to_date(sysdate-1), 'YYYY') as current_year from dual
),

day_balance as (
select year_month,country,le_book,Sequence_FD,balance as balance,column_,dr_cr_bal_ind,bal_type
    from VISIONRW.Fin_Dly_Balances 
    unpivot ( 
        balance for column_
        in (balance_01, balance_02, balance_03, balance_04, balance_05, balance_06, balance_07, balance_08, balance_09, balance_10, balance_11, balance_12, balance_13, balance_14, balance_15, balance_16, balance_17, balance_18, balance_19, balance_20, balance_21, balance_22, balance_23, balance_24, balance_25, balance_26, balance_27, balance_28, balance_29, balance_30, balance_31)
        ) balance_unpivot 

       where year_month between '202001' and  (select year_month from dates fetch next 1 row only)
),

default_pl as (SELECT
    fin_dly_headers.year_month             AS year_month,
    frl_lines_pl.frl_attribute_04          AS frl_attribute_04,
    ( (
        SELECT
            frl_attribute_description
        FROM
            visionrw.frl_attributes
        WHERE
            frl_attribute_level = 4
            AND frl_lines_pl.frl_attribute_04 = frl_attribute
    ) ) AS frl_att_04_description,
    frl_lines_pl.frl_attribute_02          AS frl_attribute_02,
    ( (
        SELECT
            frl_attribute_description
        FROM
            visionrw.frl_attributes
        WHERE
            frl_attribute_level = 2
            AND frl_lines_pl.frl_attribute_02 = frl_attribute
    ) ) AS frl_att_02_description,
    frl_lines_pl.frl_attribute_06          AS frl_attribute_06,
    ( (
        SELECT
            frl_attribute_description
        FROM
            visionrw.frl_attributes
        WHERE
            frl_attribute_level = 6
            AND frl_lines_pl.frl_attribute_06 = frl_attribute
    ) ) AS frl_att_06_description,
    frl_lines_bs.frl_attribute_01          AS frl_attribute_01,
    ( (
        SELECT
            frl_attribute_description
       FROM
            visionrw.frl_attributes
        WHERE
            frl_attribute_level = 1
            AND frl_lines_bs.frl_attribute_01 = frl_attribute
    ) ) AS frl_att_01_description,
    product_expanded.product_description   AS product_description,
    frl_lines_bs.source_type               AS source_type,
    fin_dly_headers.customer_id            AS customer_id,
    fin_dly_headers.bs_gl                  AS bs_gl,
    fin_dly_headers.pl_gl                  AS pl_gl,
    fin_dly_headers.vision_sbu             AS vision_sbu,
    fin_dly_headers.currency               AS currency,
        accounts_dly.ACCOUNT_TYPE               AS ACCOUNT_TYPE,    
        accounts_dly.scheme_code               AS scheme_code,
    fin_dly_headers.record_type            AS record_type,
    TO_CHAR(fin_dly_headers.date_last_modified, 'YYYY-MM-DD') as date_last_modified,
    fin_dly_balances.bal_type              AS bal_type,
    to_number(SUM(fin_dly_balances.balance)) AS balance 

FROM
    visionrw.fin_dly_headers             fin_dly_headers
    JOIN day_balance            fin_dly_balances ON fin_dly_headers.country = fin_dly_balances.country
                                              AND fin_dly_headers.le_book = fin_dly_balances.le_book
                                              AND fin_dly_headers.year_month = fin_dly_balances.year_month
                                              AND fin_dly_headers.sequence_fd = fin_dly_balances.sequence_fd
     
                                              AND fin_dly_headers.record_type != 9999
        LEFT JOIN visionrw.accounts_dly_view           accounts_dly ON fin_dly_headers.country = accounts_dly.country
                                                AND fin_dly_headers.le_book = accounts_dly.le_book
                                                AND (
        CASE
            WHEN fin_dly_headers.contract_id = '0' THEN
                fin_dly_headers.office_account
            ELSE
                fin_dly_headers.contract_id
        END
    ) = accounts_dly.account_no
                                                AND fin_dly_headers.record_type != 9999
    LEFT JOIN visionrw.ouc_expanded                ouc_expanded ON ouc_expanded.vision_ouc = fin_dly_headers.vision_ouc
                                           AND fin_dly_headers.record_type != 9999
    LEFT JOIN visionrw.vw_customers_dly_expanded   customers_dly_expanded ON customers_dly_expanded.country = fin_dly_headers.country
                                                                  AND customers_dly_expanded.le_book = fin_dly_headers.le_book
                                                                  AND customers_dly_expanded.customer_id = fin_dly_headers.customer_id
                                                                  AND fin_dly_headers.record_type != 9999
    JOIN visionrw.fin_dly_mappings            fin_dly_mappings ON fin_dly_balances.country = fin_dly_mappings.country
                                              AND fin_dly_balances.le_book = fin_dly_mappings.le_book
                                              AND fin_dly_balances.year_month = fin_dly_mappings.year_month
                                              AND fin_dly_balances.sequence_fd = fin_dly_mappings.sequence_fd
                                              AND fin_dly_balances.dr_cr_bal_ind = fin_dly_mappings.dr_cr_bal_ind
    JOIN visionrw.product_expanded            product_expanded ON product_expanded.product = fin_dly_mappings.product
    JOIN visionrw.frl_lines                   frl_lines_bs ON fin_dly_mappings.frl_line_bs = frl_lines_bs.frl_line
    JOIN visionrw.frl_lines                   frl_lines_pl ON fin_dly_mappings.frl_line_pl = frl_lines_pl.frl_line
WHERE
    fin_dly_headers.year_month between '202001' and  (select year_month from dates fetch next 1 row only)
    AND frl_lines_bs.source_type = '0'
    AND fin_dly_headers.record_type NOT IN (
        '9999'
    )
    AND fin_dly_balances.bal_type = '54'
    and frl_lines_pl.frl_attribute_04='FA040100'
GROUP BY
    fin_dly_headers.year_month,
    frl_lines_pl.frl_attribute_04,
    frl_lines_pl.frl_attribute_02,
    frl_lines_pl.frl_attribute_06,
    frl_lines_bs.frl_attribute_01,
    product_expanded.product_description,
    frl_lines_bs.source_type,
    fin_dly_headers.customer_id,
    fin_dly_headers.bs_gl,
    fin_dly_headers.pl_gl,
    fin_dly_headers.vision_sbu,
    fin_dly_headers.currency,
    accounts_dly.ACCOUNT_TYPE,
    accounts_dly.scheme_code,
    fin_dly_headers.record_type,
    TO_CHAR(fin_dly_headers.date_last_modified, 'YYYY-MM-DD'),
    fin_dly_balances.bal_type)
    select * from default_pl
"""
income = pd.read_sql(query_inc, con=conn)
income_csv = income.to_csv(r'C:\Users\clement nshimiyima\Desktop\dt\income.csv',index=None,header=True)   

In [6]:
# Expenses
query_ex="""
with dates as (
select LPAD(extract(day from to_date(sysdate-1)), 2, '0') as day,TO_CHAR(to_date(sysdate-1), 'YYYYMM') as year_month , TO_CHAR(to_date(sysdate-1), 'MM') as current_month,
TO_CHAR(to_date(sysdate-1), 'YYYY') as current_year from dual
),

day_balance as (
select year_month,country,le_book,Sequence_FD,balance as balance,column_,dr_cr_bal_ind,bal_type
    from VISIONRW.Fin_Dly_Balances 
    unpivot ( 
        balance for column_
        in (balance_01, balance_02, balance_03, balance_04, balance_05, balance_06, balance_07, balance_08, balance_09, balance_10, balance_11, balance_12, balance_13, balance_14, balance_15, balance_16, balance_17, balance_18, balance_19, balance_20, balance_21, balance_22, balance_23, balance_24, balance_25, balance_26, balance_27, balance_28, balance_29, balance_30, balance_31)
        ) balance_unpivot 

       where year_month between '202001' and  (select year_month from dates fetch next 1 row only)
),

default_pl as (SELECT

    fin_dly_headers.year_month             AS year_month,
    frl_lines_pl.frl_attribute_04          AS frl_attribute_04,
    ( (
        SELECT
            frl_attribute_description
        FROM
            visionrw.frl_attributes
        WHERE
            frl_attribute_level = 4
            AND frl_lines_pl.frl_attribute_04 = frl_attribute
    ) ) AS frl_att_04_description,
    frl_lines_pl.frl_attribute_02          AS frl_attribute_02,
    ( (
        SELECT
            frl_attribute_description
        FROM
            visionrw.frl_attributes
        WHERE
            frl_attribute_level = 2
            AND frl_lines_pl.frl_attribute_02 = frl_attribute
    ) ) AS frl_att_02_description,
    frl_lines_pl.frl_attribute_06          AS frl_attribute_06,
    ( (
        SELECT
            frl_attribute_description
        FROM
            visionrw.frl_attributes
        WHERE
            frl_attribute_level = 6
            AND frl_lines_pl.frl_attribute_06 = frl_attribute
    ) ) AS frl_att_06_description,
    frl_lines_bs.frl_attribute_01          AS frl_attribute_01,
    ( (
        SELECT
            frl_attribute_description
       FROM
            visionrw.frl_attributes
        WHERE
            frl_attribute_level = 1
            AND frl_lines_bs.frl_attribute_01 = frl_attribute
    ) ) AS frl_att_01_description,
    product_expanded.product_description   AS product_description,
    frl_lines_bs.source_type               AS source_type,
    fin_dly_headers.customer_id            AS customer_id,
    fin_dly_headers.bs_gl                  AS bs_gl,
    fin_dly_headers.pl_gl                  AS pl_gl,
    fin_dly_headers.vision_sbu             AS vision_sbu,
    fin_dly_headers.currency               AS currency,
        accounts_dly.ACCOUNT_TYPE               AS ACCOUNT_TYPE,    
        accounts_dly.scheme_code               AS scheme_code,
    fin_dly_headers.record_type            AS record_type,
    TO_CHAR(fin_dly_headers.date_last_modified, 'YYYY-MM-DD') as date_last_modified,    
    fin_dly_balances.bal_type              AS bal_type,
    to_number(SUM(fin_dly_balances.balance)) AS balance 

FROM
    visionrw.fin_dly_headers             fin_dly_headers
    JOIN day_balance            fin_dly_balances ON fin_dly_headers.country = fin_dly_balances.country
                                              AND fin_dly_headers.le_book = fin_dly_balances.le_book
                                              AND fin_dly_headers.year_month = fin_dly_balances.year_month
                                              AND fin_dly_headers.sequence_fd = fin_dly_balances.sequence_fd
                                              AND fin_dly_headers.record_type != 9999
        LEFT JOIN visionrw.accounts_dly_view           accounts_dly ON fin_dly_headers.country = accounts_dly.country
                                                AND fin_dly_headers.le_book = accounts_dly.le_book
                                                AND (
        CASE
            WHEN fin_dly_headers.contract_id = '0' THEN
                fin_dly_headers.office_account
            ELSE
                fin_dly_headers.contract_id
        END
    ) = accounts_dly.account_no
                                               AND fin_dly_headers.record_type != 9999
    LEFT JOIN visionrw.ouc_expanded                ouc_expanded ON ouc_expanded.vision_ouc = fin_dly_headers.vision_ouc
                                           AND fin_dly_headers.record_type != 9999
    LEFT JOIN visionrw.vw_customers_dly_expanded   customers_dly_expanded ON customers_dly_expanded.country = fin_dly_headers.country
                                                                  AND customers_dly_expanded.le_book = fin_dly_headers.le_book
                                                                  AND customers_dly_expanded.customer_id = fin_dly_headers.customer_id
                                                                  AND fin_dly_headers.record_type != 9999
    JOIN visionrw.fin_dly_mappings            fin_dly_mappings ON fin_dly_balances.country = fin_dly_mappings.country
                                              AND fin_dly_balances.le_book = fin_dly_mappings.le_book
                                              AND fin_dly_balances.year_month = fin_dly_mappings.year_month
                                              AND fin_dly_balances.sequence_fd = fin_dly_mappings.sequence_fd
                                              AND fin_dly_balances.dr_cr_bal_ind = fin_dly_mappings.dr_cr_bal_ind
    JOIN visionrw.product_expanded            product_expanded ON product_expanded.product = fin_dly_mappings.product
    JOIN visionrw.frl_lines                   frl_lines_bs ON fin_dly_mappings.frl_line_bs = frl_lines_bs.frl_line
    JOIN visionrw.frl_lines                   frl_lines_pl ON fin_dly_mappings.frl_line_pl = frl_lines_pl.frl_line
WHERE
    fin_dly_headers.year_month between '202001' and  (select year_month from dates fetch next 1 row only)
    AND frl_lines_bs.source_type = '0'
    AND fin_dly_headers.record_type NOT IN (
        '9999'
    )
    AND fin_dly_balances.bal_type = '54'
    and frl_lines_pl.frl_attribute_04='FA040200'
GROUP BY
    fin_dly_headers.year_month,
    frl_lines_pl.frl_attribute_04,
    frl_lines_pl.frl_attribute_02,
    frl_lines_pl.frl_attribute_06,
    frl_lines_bs.frl_attribute_01,
    product_expanded.product_description,
    frl_lines_bs.source_type,
    fin_dly_headers.customer_id,
    fin_dly_headers.bs_gl,
    fin_dly_headers.pl_gl,
    fin_dly_headers.vision_sbu,
    fin_dly_headers.currency,
    accounts_dly.ACCOUNT_TYPE,
    accounts_dly.scheme_code,
    fin_dly_headers.record_type,
    TO_CHAR(fin_dly_headers.date_last_modified, 'YYYY-MM-DD'),    
    fin_dly_balances.bal_type)
    select * from default_pl
"""    
expense = pd.read_sql(query_ex, con=conn) 
expense_csv = expense.to_csv(r'C:\Users\clement nshimiyima\Desktop\dt\expense.csv',index=None,header=True)   

In [7]:
####Fees& commissions###########################
query_fee="""
with dates as (
--  select LPAD(extract(day from to_date('20200131','yyyymmdd')), 2, '0') as day,TO_CHAR(to_date('20200131','yyyymmdd'), 'YYYYMM') as year_month from dual
select LPAD(extract(day from to_date(sysdate-1)), 2, '0') as day,TO_CHAR(to_date(sysdate-1), 'YYYYMM') as year_month , TO_CHAR(to_date(sysdate-1), 'MM') as current_month,
TO_CHAR(to_date(sysdate-1), 'YYYY') as current_year from dual
),

day_balance as (
select year_month,country,le_book,Sequence_FD,balance as balance,column_,dr_cr_bal_ind,bal_type
    from VISIONRW.Fin_Dly_Balances 
    unpivot ( 
        balance for column_
        in (balance_01, balance_02, balance_03, balance_04, balance_05, balance_06, balance_07, balance_08, balance_09, balance_10, balance_11, balance_12, balance_13, balance_14, balance_15, balance_16, balance_17, balance_18, balance_19, balance_20, balance_21, balance_22, balance_23, balance_24, balance_25, balance_26, balance_27, balance_28, balance_29, balance_30, balance_31)
        ) balance_unpivot 

       where year_month between '202001' and  (select year_month from dates fetch next 1 row only)
),

default_pl as (SELECT
    fin_dly_headers.year_month             AS year_month,
    frl_lines_pl.frl_attribute_04          AS frl_attribute_04,
    ( (
        SELECT
            frl_attribute_description
        FROM
            visionrw.frl_attributes
        WHERE
            frl_attribute_level = 4
            AND frl_lines_pl.frl_attribute_04 = frl_attribute
    ) ) AS frl_att_04_description,
    frl_lines_pl.frl_attribute_02          AS frl_attribute_02,
    ( (
        SELECT
            frl_attribute_description
        FROM
            visionrw.frl_attributes
        WHERE
            frl_attribute_level = 2
            AND frl_lines_pl.frl_attribute_02 = frl_attribute
    ) ) AS frl_att_02_description,
    frl_lines_pl.frl_attribute_06          AS frl_attribute_06,
    ( (
        SELECT
            frl_attribute_description
        FROM
            visionrw.frl_attributes
        WHERE
            frl_attribute_level = 6
            AND frl_lines_pl.frl_attribute_06 = frl_attribute
    ) ) AS frl_att_06_description,
    frl_lines_bs.frl_attribute_01          AS frl_attribute_01,
    ( (
        SELECT
            frl_attribute_description
       FROM
            visionrw.frl_attributes
        WHERE
            frl_attribute_level = 1
            AND frl_lines_bs.frl_attribute_01 = frl_attribute
    ) ) AS frl_att_01_description,
    product_expanded.product_description   AS product_description,
    frl_lines_bs.source_type               AS source_type,
    fin_dly_headers.customer_id            AS customer_id,
    fin_dly_headers.bs_gl                  AS bs_gl,
    fin_dly_headers.pl_gl                  AS pl_gl,
    fin_dly_headers.vision_sbu             AS vision_sbu,
    fin_dly_headers.currency               AS currency,
        accounts_dly.ACCOUNT_TYPE               AS ACCOUNT_TYPE,    
        accounts_dly.scheme_code               AS scheme_code,
    fin_dly_headers.record_type            AS record_type,
    fin_dly_balances.bal_type              AS bal_type,
    TO_CHAR(fin_dly_headers.date_last_modified, 'YYYY-MM-DD') AS date_last_modified,
    to_number(SUM(fin_dly_balances.balance)) AS balance 

FROM
    visionrw.fin_dly_headers             fin_dly_headers
    JOIN day_balance            fin_dly_balances ON fin_dly_headers.country = fin_dly_balances.country
                                              AND fin_dly_headers.le_book = fin_dly_balances.le_book
                                              AND fin_dly_headers.year_month = fin_dly_balances.year_month
                                              AND fin_dly_headers.sequence_fd = fin_dly_balances.sequence_fd
                                              AND fin_dly_headers.record_type != 9999
        LEFT JOIN visionrw.accounts_dly_view           accounts_dly ON fin_dly_headers.country = accounts_dly.country
                                                AND fin_dly_headers.le_book = accounts_dly.le_book
                                                AND (
        CASE
            WHEN fin_dly_headers.contract_id = '0' THEN
                fin_dly_headers.office_account
            ELSE
                fin_dly_headers.contract_id
        END
    ) = accounts_dly.account_no
                                                AND fin_dly_headers.record_type != 9999

    LEFT JOIN visionrw.ouc_expanded                ouc_expanded ON ouc_expanded.vision_ouc = fin_dly_headers.vision_ouc
                                           AND fin_dly_headers.record_type != 9999
    LEFT JOIN visionrw.vw_customers_dly_expanded   customers_dly_expanded ON customers_dly_expanded.country = fin_dly_headers.country
                                                                  AND customers_dly_expanded.le_book = fin_dly_headers.le_book
                                                                  AND customers_dly_expanded.customer_id = fin_dly_headers.customer_id
                                                                  AND fin_dly_headers.record_type != 9999
    JOIN visionrw.fin_dly_mappings            fin_dly_mappings ON fin_dly_balances.country = fin_dly_mappings.country
                                              AND fin_dly_balances.le_book = fin_dly_mappings.le_book
                                              AND fin_dly_balances.year_month = fin_dly_mappings.year_month
                                              AND fin_dly_balances.sequence_fd = fin_dly_mappings.sequence_fd
                                              AND fin_dly_balances.dr_cr_bal_ind = fin_dly_mappings.dr_cr_bal_ind
    JOIN visionrw.product_expanded            product_expanded ON product_expanded.product = fin_dly_mappings.product
    JOIN visionrw.frl_lines                   frl_lines_bs ON fin_dly_mappings.frl_line_bs = frl_lines_bs.frl_line
    JOIN visionrw.frl_lines                   frl_lines_pl ON fin_dly_mappings.frl_line_pl = frl_lines_pl.frl_line
WHERE
    fin_dly_headers.year_month between '202001' and  (select year_month from dates fetch next 1 row only)
    AND frl_lines_bs.source_type = '0'
    AND fin_dly_headers.record_type NOT IN (
        '9999'
    )
    AND fin_dly_balances.bal_type = '54'
    and frl_lines_pl.frl_attribute_04='FA040300'
GROUP BY
    fin_dly_headers.year_month,
    frl_lines_pl.frl_attribute_04,
    frl_lines_pl.frl_attribute_02,
    frl_lines_pl.frl_attribute_06,
    frl_lines_bs.frl_attribute_01,
    product_expanded.product_description,
    frl_lines_bs.source_type,
    fin_dly_headers.customer_id,
    fin_dly_headers.bs_gl,
    fin_dly_headers.pl_gl,
    fin_dly_headers.vision_sbu,
    fin_dly_headers.currency,
    accounts_dly.ACCOUNT_TYPE,
    accounts_dly.scheme_code,
    fin_dly_headers.record_type,
    TO_CHAR(fin_dly_headers.date_last_modified, 'YYYY-MM-DD'),
    fin_dly_balances.bal_type)
    select * from default_pl
"""
fees = pd.read_sql(query_fee, con=get_connection())
fees.to_csv(r'C:\Users\clement nshimiyima\Desktop\dt\fees.csv',index=None,header=True)

In [8]:
# Loans and deposits current date
query="""
with dates as (
select LPAD(extract(day from to_date(sysdate-1)), 2, '0') as day,TO_CHAR(to_date(sysdate-1), 'YYYYMM') as year_month , TO_CHAR(to_date(sysdate-1), 'MM') as current_month,
TO_CHAR(to_date(sysdate-1), 'YYYY') as current_year from dual
),

day_balance as (
select year_month,country,le_book,Sequence_FD,to_number(balance) as balance,dr_cr_bal_ind,bal_type
    from VISIONRW.Fin_Dly_Balances 
    unpivot ( 
        balance for column_
        in (balance_01, balance_02, balance_03, balance_04, balance_05, balance_06, balance_07, balance_08, balance_09, balance_10, balance_11, balance_12, balance_13, balance_14, balance_15, balance_16, balance_17, balance_18, balance_19, balance_20, balance_21, balance_22, balance_23, balance_24, balance_25, balance_26, balance_27, balance_28, balance_29, balance_30, balance_31)
        ) balance_unpivot 
   where year_month = (select year_month from dates fetch next 1 row only) and column_ = CONCAT('BALANCE_',(select day from dates fetch next 1 row only))

),
day_rate as (
select year_month,country,le_book,currency,rate as rate
    from VISIONRW.Currency_Rates_Daily
    unpivot ( 
        rate for column_
        in (rate_01, rate_02, rate_03, rate_04, rate_05, rate_06, rate_07, rate_08, rate_09, rate_10, rate_11, rate_12, rate_13, rate_14, rate_15, rate_16, rate_17, rate_18, rate_19, rate_20, rate_21, rate_22, rate_23, rate_24, rate_25, rate_26, rate_27, rate_28, rate_29, rate_30, rate_31)
        ) rate_unpivot 
   where year_month = (select year_month from dates fetch next 1 row only) 
   and column_ = CONCAT('RATE_',(select day from dates fetch next 1 row only)) 
),

default_data as (
SELECT  Fin_Dly_Headers.Country AS Country,
    Fin_Dly_Headers.LE_Book AS LE_Book ,
    Fin_Dly_Headers.Year_Month AS Year_Month ,
    FRL_Lines_BS.FRL_Attribute_05 AS FRL_Attribute_05 ,
((select FRL_ATTRIBUTE_DESCRIPTION from  VISIONRW.FRL_ATTRIBUTES WHERE FRL_ATTRIBUTE_LEVEL =5 
AND FRL_Lines_BS.FRL_ATTRIBUTE_05 = FRL_ATTRIBUTE))  AS FRL_Att_05_Description,
    FRL_Lines_BS.FRL_Attribute_01 AS FRL_Attribute_01 ,
((select FRL_ATTRIBUTE_DESCRIPTION from  VISIONRW.FRL_ATTRIBUTES WHERE FRL_ATTRIBUTE_LEVEL =1 
AND FRL_Lines_BS.FRL_ATTRIBUTE_01 = FRL_ATTRIBUTE))  AS FRL_Att_01_Description, 
    FRL_Lines_BS.FRL_Attribute_03 AS FRL_Attribute_03 , 
    Customers_Dly_Expanded.Customer_Id AS Customer_Id ,
    Fin_Dly_Headers.Vision_SBU AS Vision_SBU , 
Fin_Dly_Headers.Record_Type AS Record_Type , Fin_Dly_Balances.Bal_Type AS Bal_Type, 
Fin_Dly_Headers.Currency AS Currency ,TRIM(TO_CHAR(Currency_Rates_Daily.rate,'999,999,999,999,999,990.00000')) AS rate,
TRIM(TO_CHAR(SUM(Fin_Dly_Balances.balance) ,'999,999,999,999,999,990.00')) AS balance
FROM  VISIONRW.Fin_Dly_Headers  Fin_Dly_Headers
 JOIN day_balance  Fin_Dly_Balances ON 
    Fin_Dly_Headers.Country = Fin_Dly_Balances.Country AND 
    Fin_Dly_Headers.Le_Book = Fin_Dly_Balances.Le_Book AND 
    Fin_Dly_Headers.Year_Month = Fin_Dly_Balances.Year_Month AND 
    Fin_Dly_Headers.Sequence_FD = Fin_Dly_Balances.Sequence_FD And 
    Fin_Dly_Headers.RECORD_TYPE != 9999
 JOIN day_rate  Currency_Rates_Daily ON 
 Fin_Dly_Headers.Country = Currency_Rates_Daily.Country AND 
 Fin_Dly_Headers.Le_Book = Currency_Rates_Daily.Le_Book AND
 Fin_Dly_Headers.Year_Month = Currency_Rates_Daily.Year_Month 
 AND Fin_Dly_Headers.currency = Currency_Rates_Daily.currency And Fin_Dly_Headers.RECORD_TYPE != 9999
 LEFT JOIN VISIONRW.VW_Customers_Dly_Expanded  
 Customers_Dly_Expanded ON Customers_Dly_Expanded.Country = Fin_Dly_Headers.Country 
 AND Customers_Dly_Expanded.Le_Book = Fin_Dly_Headers.Le_Book 
 AND Customers_Dly_Expanded.Customer_ID = Fin_Dly_Headers.Customer_ID And
 Fin_Dly_Headers.RECORD_TYPE != 9999
 JOIN VISIONRW.Fin_Dly_Mappings  Fin_Dly_Mappings ON 
 Fin_Dly_Balances.Country = Fin_Dly_Mappings.Country 
 AND Fin_Dly_Balances.Le_Book = Fin_Dly_Mappings.Le_Book 
 AND Fin_Dly_Balances.Year_Month = Fin_Dly_Mappings.Year_Month AND 
 Fin_Dly_Balances.Sequence_FD = Fin_Dly_Mappings.Sequence_FD AND 
 Fin_Dly_Balances.Dr_Cr_Bal_Ind = Fin_Dly_Mappings.Dr_Cr_Bal_Ind
 JOIN VISIONRW.MRL_Lines  MRL_Lines ON Fin_Dly_Mappings.MRL_LINE = MRL_Lines.MRL_LINE
 JOIN VISIONRW.FRL_Lines  FRL_Lines_BS ON Fin_Dly_Mappings.FRL_Line_BS = FRL_Lines_BS.FRL_Line  
WHERE  Fin_Dly_Headers.Country = 'RW'
    AND  Fin_Dly_Headers.LE_Book = '01'
    AND  FRL_Lines_BS.FRL_Attribute_05 in  ('FA051050','FA052010' ) 
    AND  FRL_Lines_BS.FRL_Attribute_03 != 'FA031105' 
    AND  Fin_Dly_Headers.Record_Type not in  ('999' ) 
    AND  Fin_Dly_Balances.Bal_Type = '51' 
    AND Fin_Dly_Headers.customer_id<>'DOOO'
GROUP BY  Fin_Dly_Headers.Country , Fin_Dly_Headers.LE_Book ,
FRL_Lines_BS.FRL_Attribute_01,
Fin_Dly_Headers.Year_Month , FRL_Lines_BS.FRL_Attribute_05 , 
FRL_Lines_BS.FRL_Attribute_03,
Customers_Dly_Expanded.Customer_Id ,
Fin_Dly_Headers.Vision_SBU ,
Fin_Dly_Headers.Record_Type , Fin_Dly_Balances.Bal_Type,
Fin_Dly_Headers.Currency , Currency_Rates_Daily.rate 
)
SELECT * FROM default_data 
"""
PL= pd.read_sql(query_fee, con=conn)
PL.to_csv(r'C:\Users\clement nshimiyima\Desktop\dt\PL.csv',index=None,header=True)

In [9]:
# Loans vs deposits overtime#########
query_bs="""
SELECT 
VISIONRW.Fin_Dly_Headers.Year_Month AS Year_Month ,VISIONRW.FRL_Lines_BS.FRL_Attribute_05 AS FRL_Attribute_05 ,
((select FRL_ATTRIBUTE_DESCRIPTION from  VISIONRW.FRL_ATTRIBUTES WHERE FRL_ATTRIBUTE_LEVEL =5 
AND FRL_Lines_BS.FRL_ATTRIBUTE_05 = FRL_ATTRIBUTE))  AS FRL_Att_05_Description,
((select FRL_ATTRIBUTE_DESCRIPTION from  VISIONRW.FRL_ATTRIBUTES WHERE FRL_ATTRIBUTE_LEVEL =1 AND 
VISIONRW.FRL_Lines_BS.FRL_ATTRIBUTE_01 = FRL_ATTRIBUTE))  AS FRL_Att_01_Description ,
VISIONRW.Fin_Dly_Headers.Customer_Id AS Customer_Id,
VISIONRW.OUC_Expanded.OUC_Description AS OUC_Description,
VISIONRW.Fin_Dly_Headers.Vision_SBU AS Vision_SBU, 
VISIONRW.Fin_Dly_Headers.Record_Type AS Record_Type , 
VISIONRW.Fin_Dly_Balances.Bal_Type AS Bal_Type , 
VISIONRW.Fin_Dly_Headers.Currency AS Currency,
SUM(VISIONRW.Fin_Dly_Balances.Balance_01*VISIONRW.Currency_Rates_Daily.RATE_01) AS Balance_01,
SUM(VISIONRW.Fin_Dly_Balances.Balance_02*VISIONRW.Currency_Rates_Daily.RATE_02) AS Balance_02,
SUM(VISIONRW.Fin_Dly_Balances.Balance_03*VISIONRW.Currency_Rates_Daily.RATE_03) AS Balance_03,
SUM(VISIONRW.Fin_Dly_Balances.Balance_04*VISIONRW.Currency_Rates_Daily.RATE_04) AS Balance_04,
SUM(VISIONRW.Fin_Dly_Balances.Balance_05*VISIONRW.Currency_Rates_Daily.RATE_05) AS Balance_05,
SUM(VISIONRW.Fin_Dly_Balances.Balance_06*VISIONRW.Currency_Rates_Daily.RATE_06) AS Balance_06,
SUM(VISIONRW.Fin_Dly_Balances.Balance_07*VISIONRW.Currency_Rates_Daily.RATE_07) AS Balance_07,
SUM(VISIONRW.Fin_Dly_Balances.Balance_08*VISIONRW.Currency_Rates_Daily.RATE_08) AS Balance_08,
SUM(VISIONRW.Fin_Dly_Balances.Balance_09*VISIONRW.Currency_Rates_Daily.RATE_09) AS Balance_09,
SUM(VISIONRW.Fin_Dly_Balances.Balance_10*VISIONRW.Currency_Rates_Daily.RATE_10) AS Balance_10,
SUM(VISIONRW.Fin_Dly_Balances.Balance_11*VISIONRW.Currency_Rates_Daily.RATE_11) AS Balance_11,
SUM(VISIONRW.Fin_Dly_Balances.Balance_12*VISIONRW.Currency_Rates_Daily.RATE_12) AS Balance_12,
SUM(VISIONRW.Fin_Dly_Balances.Balance_13*VISIONRW.Currency_Rates_Daily.RATE_13) AS Balance_13,
SUM(VISIONRW.Fin_Dly_Balances.Balance_14*VISIONRW.Currency_Rates_Daily.RATE_14) AS Balance_14,
SUM(VISIONRW.Fin_Dly_Balances.Balance_15*VISIONRW.Currency_Rates_Daily.RATE_15) AS Balance_15,
SUM(VISIONRW.Fin_Dly_Balances.Balance_16*VISIONRW.Currency_Rates_Daily.RATE_16) AS Balance_16,
SUM(VISIONRW.Fin_Dly_Balances.Balance_17*VISIONRW.Currency_Rates_Daily.RATE_17) AS Balance_17,
SUM(VISIONRW.Fin_Dly_Balances.Balance_18*VISIONRW.Currency_Rates_Daily.RATE_18) AS Balance_18,
SUM(VISIONRW.Fin_Dly_Balances.Balance_19*VISIONRW.Currency_Rates_Daily.RATE_19) AS Balance_19,
SUM(VISIONRW.Fin_Dly_Balances.Balance_20*VISIONRW.Currency_Rates_Daily.RATE_20) AS Balance_20,
SUM(VISIONRW.Fin_Dly_Balances.Balance_21*VISIONRW.Currency_Rates_Daily.RATE_21) AS Balance_21,
SUM(VISIONRW.Fin_Dly_Balances.Balance_22*VISIONRW.Currency_Rates_Daily.RATE_22) AS Balance_22,
SUM(VISIONRW.Fin_Dly_Balances.Balance_23*VISIONRW.Currency_Rates_Daily.RATE_23) AS Balance_23,
SUM(VISIONRW.Fin_Dly_Balances.Balance_24*VISIONRW.Currency_Rates_Daily.RATE_24) AS Balance_24,
SUM(VISIONRW.Fin_Dly_Balances.Balance_25*VISIONRW.Currency_Rates_Daily.RATE_25) AS Balance_25,
SUM(VISIONRW.Fin_Dly_Balances.Balance_26*VISIONRW.Currency_Rates_Daily.RATE_26) AS Balance_26,
SUM(VISIONRW.Fin_Dly_Balances.Balance_27*VISIONRW.Currency_Rates_Daily.RATE_27) AS Balance_27,
SUM(VISIONRW.Fin_Dly_Balances.Balance_28*VISIONRW.Currency_Rates_Daily.RATE_28) AS Balance_28,
SUM(VISIONRW.Fin_Dly_Balances.Balance_29*VISIONRW.Currency_Rates_Daily.RATE_29) AS Balance_29,
SUM(VISIONRW.Fin_Dly_Balances.Balance_30*VISIONRW.Currency_Rates_Daily.RATE_30) AS Balance_30,
SUM(VISIONRW.Fin_Dly_Balances.Balance_31*VISIONRW.Currency_Rates_Daily.RATE_31) AS Balance_31
FROM 
VISIONRW.Fin_Dly_Headers Fin_Dly_Headers
JOIN VISIONRW.Fin_Dly_Balances  Fin_Dly_Balances ON 
VISIONRW.Fin_Dly_Headers.Country = VISIONRW.Fin_Dly_Balances.Country AND 
VISIONRW.Fin_Dly_Headers.Le_Book = VISIONRW.Fin_Dly_Balances.Le_Book AND 
VISIONRW.Fin_Dly_Headers.Year_Month = VISIONRW.Fin_Dly_Balances.Year_Month AND 
VISIONRW.Fin_Dly_Headers.Sequence_FD = VISIONRW.Fin_Dly_Balances.Sequence_FD And 
VISIONRW.Fin_Dly_Headers.RECORD_TYPE != 9999
JOIN VISIONRW.Currency_Rates_Daily ON 
 VISIONRW.Fin_Dly_Headers.Country = VISIONRW.Currency_Rates_Daily.Country AND 
 VISIONRW.Fin_Dly_Headers.Le_Book = VISIONRW.Currency_Rates_Daily.Le_Book AND
 VISIONRW.Fin_Dly_Headers.Year_Month = VISIONRW.Currency_Rates_Daily.Year_Month 
 AND VISIONRW.Fin_Dly_Headers.currency = VISIONRW.Currency_Rates_Daily.currency 
 And VISIONRW.Fin_Dly_Headers.RECORD_TYPE != 9999
LEFT JOIN VISIONRW.OUC_Expanded  OUC_Expanded ON
VISIONRW.Ouc_Expanded.Vision_OUC = VISIONRW.Fin_Dly_Headers.Vision_OUC 
And VISIONRW.Fin_Dly_Headers.RECORD_TYPE != 9999
LEFT JOIN VISIONRW.VW_Customers_Dly_Expanded  Customers_Dly_Expanded ON
VISIONRW.Customers_Dly_Expanded.Country = VISIONRW.Fin_Dly_Headers.Country AND 
VISIONRW.Customers_Dly_Expanded.Le_Book = VISIONRW.Fin_Dly_Headers.Le_Book AND 
VISIONRW.Customers_Dly_Expanded.Customer_ID = VISIONRW.Fin_Dly_Headers.Customer_ID And 
VISIONRW.Fin_Dly_Headers.RECORD_TYPE != 9999
JOIN VISIONRW.Fin_Dly_Mappings  Fin_Dly_Mappings ON 
VISIONRW.Fin_Dly_Balances.Country = VISIONRW.Fin_Dly_Mappings.Country AND
VISIONRW.Fin_Dly_Balances.Le_Book = VISIONRW.Fin_Dly_Mappings.Le_Book AND 
VISIONRW.Fin_Dly_Balances.Year_Month = VISIONRW.Fin_Dly_Mappings.Year_Month AND
VISIONRW.Fin_Dly_Balances.Sequence_FD = VISIONRW.Fin_Dly_Mappings.Sequence_FD AND 
VISIONRW.Fin_Dly_Balances.Dr_Cr_Bal_Ind = VISIONRW.Fin_Dly_Mappings.Dr_Cr_Bal_Ind
JOIN VISIONRW.FRL_Lines  FRL_Lines_BS ON 
VISIONRW.Fin_Dly_Mappings.FRL_Line_BS = VISIONRW.FRL_Lines_BS.FRL_Line  
WHERE VISIONRW.FRL_Lines_BS.FRL_Attribute_05 in  ('FA051050','FA052010') AND  
VISIONRW.FRL_Lines_BS.Source_Type = '0' AND  VISIONRW.Fin_Dly_Headers.Record_Type not in  ('9999' ) AND
VISIONRW.Fin_Dly_Balances.Bal_Type = '51' AND VISIONRW.Fin_Dly_Headers.CUSTOMER_ID<>'DOOO' 
AND VISIONRW.Fin_Dly_Headers.Year_Month>=202003
GROUP BY 
VISIONRW.Fin_Dly_Headers.Year_Month ,     
VISIONRW.FRL_Lines_BS.FRL_Attribute_05,
 VISIONRW.FRL_Lines_BS.FRL_Attribute_01,   
VISIONRW.Fin_Dly_Headers.Customer_Id ,  
VISIONRW.OUC_Expanded.OUC_Description , 
VISIONRW.Fin_Dly_Headers.Vision_SBU ,
VISIONRW.Fin_Dly_Headers.Record_Type , 
VISIONRW.Fin_Dly_Balances.Bal_Type ,
VISIONRW.Fin_Dly_Headers.Currency
"""
overd = pd.read_sql(query_bs, con=conn)
overd.set_index(['YEAR_MONTH','CURRENCY','FRL_ATTRIBUTE_05','FRL_ATT_05_DESCRIPTION','FRL_ATT_01_DESCRIPTION',
                 'CUSTOMER_ID','OUC_DESCRIPTION','VISION_SBU','RECORD_TYPE','BAL_TYPE'],inplace=True)# Set yearmonth and currency as index
over = overd.stack()#To transpose a dataframe
over=pd.DataFrame(over)
loan_depos=over.reset_index() # To reset index
loan_depos=loan_depos.rename(columns={0: 'BALANCE','level_10':'DAY'})# to rename columns
d = {'BALANCE_01':1, 'BALANCE_02':2, 'BALANCE_03':3, 'BALANCE_04':4,'BALANCE_05':5, 'BALANCE_06':6,
 'BALANCE_07':7, 'BALANCE_08':8,'BALANCE_09':9, 'BALANCE_10':10, 'BALANCE_11':11, 'BALANCE_12':12,
 'BALANCE_13':13, 'BALANCE_14':14, 'BALANCE_15':15, 'BALANCE_16':16,'BALANCE_17':17, 'BALANCE_18':18,
 'BALANCE_19':19, 'BALANCE_20':20,'BALANCE_21':21, 'BALANCE_22':22, 'BALANCE_23':23, 'BALANCE_24':24,
 'BALANCE_25':25,'BALANCE_26':26, 'BALANCE_27':27, 'BALANCE_28':28, 'BALANCE_29':29,'BALANCE_30':30, 'BALANCE_31':31 }# change rate to corresponding day
loan_depos['DAY_'] = loan_depos.DAY.map(d) # map day
loan_depos['BALANCE_YEAR'] = loan_depos.YEAR_MONTH.astype(str).str[:4].astype(int)# To change year month in years
loan_depos['BALANCE_MONTH'] = loan_depos.YEAR_MONTH.astype(str).str[4:].astype(int)# To change year month in months
loan_depos['DROP_MONTHS'] = np.where(((loan_depos.BALANCE_MONTH.isin([4,6,9,11]))&(loan_depos.DAY_>30)),'Y','N')
# 	loan_depos = loan_depos[loan_depos.DROP_FEB != 'Y']
loan_depos = loan_depos[loan_depos.DROP_MONTHS != 'Y']  
loan_depos['VALUE_DATE'] = loan_depos.apply(lambda row: datetime.strptime(f"{(row.BALANCE_YEAR)}-{(row.BALANCE_MONTH)}-{(row.DAY_)}", '%Y-%m-%d'), axis=1)
loan_depos['VALUE_DATE']=loan_depos['VALUE_DATE'].astype(str)
loan_depos = loan_depos[['CUSTOMER_ID','YEAR_MONTH','VALUE_DATE','CURRENCY','BALANCE','FRL_ATTRIBUTE_05',
                         'FRL_ATT_05_DESCRIPTION','FRL_ATT_01_DESCRIPTION','OUC_DESCRIPTION','VISION_SBU']]
loan_depos_csv = loan_depos.to_csv(r'C:\Users\clement nshimiyima\Desktop\dt\loan_depos.csv',index=None,header=True)

In [53]:
# Savings